In [19]:
import pulp

In [20]:
#初期設定を与える
#ジョブの集合
Jobs = [1,2,3,4]
#ジョブの処理時間の集合
Pro_time = [1, 93, 26, 30]
#ジョブの重要度の集合
Weights = [1, 3, 3,5]
#ジョブのリリース時間の集合
Release_time = [63, 4, 63, 99]


In [21]:
#データを作成する
def make_data(Jobs,Pro_time,Weights,Relase_time):
    """
    Jobs : list
        ジョブの集合    
    Pro_time : list
        ジョブの処理時間の集合  
    Weights : list
        ジョブの重要度の集合    
    Relase_time : list
        ジョブのリリース時間の集合  
    Returns
    -------
    J : list
    P : dict
    W : dict
    R : dict
    """
    J = Jobs   
    P = dict()
    W = dict()
    R = dict()
    for j,p,w,r in zip(Jobs,Pro_time,Weights,Relase_time):
        P[j] = p
        W[j] = w
        R[j] = r
    return J,P,W,R

class Model():
    #問題の定義
    def __init__(self,J,P,W,R) -> None:
        """
        Attributes
        ----------
        J : list
            ジョブの集合    
        P : dict
            ジョブの処理時間の集合  
        W : dict    
            ジョブの重要度の集合
        R : dict    
            ジョブのリリース時間の集合
        M : int 
            ビックMの値（最も遅いリリースから全ジョブの処理時間の和を足す）
        """
        self.prob = pulp.LpProblem(name="prob", sense=pulp.LpMinimize)
        self.J = J
        self.P = P
        self.W = W
        self.R = R
        self.M = max(R.values()) + sum(P.values())  
    
    #変数を定義する
    def make_var(self):
        """
        Returns
        -------
        X : list
            ジョブの順序に関する変数  
        C : list
            ジョブの完了時間に関する変数
        S : list
            ジョブの開始時間に関する変数
        """
        self.X = pulp.LpVariable.dicts("x", [(j,k) for j in self.J for k in self.J], cat="Binary")
        self.C = pulp.LpVariable.dicts("C", self.J, lowBound=0, cat="Continuous")
        self.S = pulp.LpVariable.dicts("s", self.J, lowBound=0, cat="Continuous")
        return self.X,self.C,self.S

    #目的関数を定義する
    def make_obj(self):
        self.prob += pulp.lpSum([self.W[j] * self.C[j] for j in self.J])
        return self.prob
    
    #制約条件を定義する
    def make_const(self):
        """
        制約1 : C[j] == S[j] + P[j]
            ジョブjの完了時間は開始時間と処理時間の和
        制約2 : S[j] >= R[j]
            ジョブjの開始時間はリリース時間以上
        制約3 : C[j] <= S[k] + M * (1 - X[j,k])
            ジョブjがジョブkよりも早く終わる時、ジョブjの完了時間はジョブkの開始時間よりも早い
        制約4 : X[j,k] + X[k,j] == 1
            ジョブjとジョブkの順序は1つのみ
        """
        #制約条件を定義する
        for j in self.J:
            #制約1
            self.prob += self.C[j] == self.S[j] + self.P[j]
            #制約2
            self.prob += self.S[j] >= self.R[j]
            for k in self.J:
                #制約3
                self.prob += self.S[k] + self.M * (1 - self.X[j,k]) >= self.C[j]
                #制約4
                if j != k:
                    self.prob += self.X[j,k] + self.X[k,j] == 1 
        return self.prob

In [22]:
J,P,W,R = make_data(Jobs,Pro_time,Weights,Release_time)

In [23]:
model = Model(J,P,W,R)
model.make_var()
model.make_obj()
model.make_const()
model.prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/shukitakeuchi/Library/Caches/pypoetry/virtualenvs/job-scheduler-KeT9Yqdu-py3.8/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/q6/yv767t9s35q4h2jldrxtfq1c0000gn/T/7858e6833b554b728f313732ffb7a325-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/q6/yv767t9s35q4h2jldrxtfq1c0000gn/T/7858e6833b554b728f313732ffb7a325-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 41 COLUMNS
At line 162 RHS
At line 199 BOUNDS
At line 216 ENDATA
Problem MODEL has 36 rows, 24 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1267 - 0.00 seconds
Cgl0004I processed model has 12 rows, 10 columns (6 integer (6 of which binary)) and 36 elements
Cbc0038I Initial state - 6 integers unsatisfied sum - 1.61044
Cbc0038I Pass   1: suminf.    0.40161 (1) obj.

1

In [24]:
#modelの制約の数
print("制約の数 : ",model.prob.numConstraints())
#modelの変数の数
print("変数の数 : ",model.prob.numVariables())
#最適解
print("最適解 : ",pulp.value(model.prob.objective))
#最適解の時の変数の値
for v in model.prob.variables():
    print(v.name, "=", v.varValue)
    

制約の数 :  36
変数の数 :  24
最適解 :  1499.0
C_1 = 98.0
C_2 = 97.0
C_3 = 155.0
C_4 = 129.0
s_1 = 97.0
s_2 = 4.0
s_3 = 129.0
s_4 = 99.0
x_(1,_1) = 0.0
x_(1,_2) = 0.0
x_(1,_3) = 1.0
x_(1,_4) = 1.0
x_(2,_1) = 1.0
x_(2,_2) = 0.0
x_(2,_3) = 1.0
x_(2,_4) = 1.0
x_(3,_1) = 0.0
x_(3,_2) = 0.0
x_(3,_3) = 0.0
x_(3,_4) = 0.0
x_(4,_1) = 0.0
x_(4,_2) = 0.0
x_(4,_3) = 1.0
x_(4,_4) = 0.0


In [25]:
#pulpの計算結果を取得する
print("最適値は",model.prob.objective.value())
for j in J:
    print("ジョブ",j,"の開始時間は",pulp.value(model.S[j]))
    print("ジョブ",j,"の完了時間は",pulp.value(model.C[j]))
    print("")

最適値は 1499.0
ジョブ 1 の開始時間は 97.0
ジョブ 1 の完了時間は 98.0

ジョブ 2 の開始時間は 4.0
ジョブ 2 の完了時間は 97.0

ジョブ 3 の開始時間は 129.0
ジョブ 3 の完了時間は 155.0

ジョブ 4 の開始時間は 99.0
ジョブ 4 の完了時間は 129.0

